In [5]:
import pandas as pd
import numpy as np
import quandl
from matplotlib import pyplot 
quandl.ApiConfig.api_key = 'rzCd-JTxZsmE7G2Lidqq'

In [6]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

#### Get closing prices (up to the most recent prices available in the free WIKI database)

In [7]:
ticker_symbols = ["WIKI/AAPL","WIKI/AMZN","WIKI/MSFT","WIKI/FB","WIKI/GOOGL","WIKI/NFLX"]
closing_prices = pd.DataFrame()

def add_closing_price(df, ticker, col = 'Close'):
    new_df = pd.concat([df, 
        quandl.get(ticker, start_date="2014-12-31", end_date="2018-03-27", collapse="daily")[col]
        .rename(ticker)], axis=1)
    return new_df
        
for tckr in ticker_symbols:
    closing_prices = add_closing_price(closing_prices, tckr)

#ticker_symbols += ["SSE/TL0"]
#closing_prices = add_closing_price(closing_prices, "SSE/TL0", "Last")

closing_prices.tail(10)

,WIKI/AAPL,WIKI/AMZN,WIKI/MSFT,WIKI/FB,WIKI/GOOGL,WIKI/NFLX
2018-03-14,178.440,1591.00,93.85,184.19,1148.89,321.55
2018-03-15,178.650,1582.32,94.18,183.86,1150.61,321.09
2018-03-16,178.020,1571.68,94.60,185.09,1134.42,318.45
2018-03-19,175.300,1544.93,92.89,172.56,1100.07,313.48
2018-03-20,175.240,1586.51,93.13,168.15,1095.80,317.50
2018-03-21,171.270,1581.86,92.48,169.39,1094.00,316.48
2018-03-22,168.845,1544.10,89.79,164.89,1053.15,306.70
2018-03-23,164.940,1495.56,87.18,159.39,1026.55,300.94
2018-03-26,172.770,1555.86,93.78,160.06,1054.09,320.35
2018-03-27,168.340,1497.05,89.47,152.19,1006.94,300.69


#### Crunch some numbers, plot some graphs

In [8]:
percent_change = closing_prices.pct_change()
expected_return = percent_change.mean()
cov_return = closing_prices.pct_change().cov()
corr_return = closing_prices.pct_change().corr()


In [9]:
weights = np.random.random(len(ticker_symbols))
weights /= np.sum(weights)
np.dot(weights, expected_return)

0.0011102641024386746

In [10]:
expected_return

WIKI/AAPL     0.000626
WIKI/AMZN     0.002097
WIKI/MSFT     0.000914
WIKI/FB       0.000946
WIKI/GOOGL    0.000890
WIKI/NFLX     0.001560
dtype: float64

In [11]:
p = figure(plot_width=900, plot_height=300, x_axis_type='datetime')
colors = ['red', 'green', 'blue', 'purple', 'orange', 'yellow', 'brown', 'pink', 'black'] # random bunch of colors
color_index = 0

for ticker in expected_return.index:
    p.line(percent_change.index, percent_change[ticker].values,  \
           color=colors[color_index % len(colors)], legend=ticker)
    color_index += 1
    
p.legend.location = "bottom_right"
p.legend.click_policy="hide"
show(p)

#### Feel free to zoom/move around. The default plot is ruined by Netflix's stock split in July 2015 which produced an 86% drop. I'm not going back to 2014 since the SSE database doesn't have data for Tesla at that time and I wanted to test Andy Xia's take on Elon Musk, but if we did we would have seen something similar happen to Apple's stock in June 2014. 

In [12]:
# Record portfolio distributions used in the simulation along with their expected returns and volatility
allocations = [] 
returns = []
std_devs = []
num_portfolios = 15000

# See https://www.investopedia.com/walkthrough/corporate-finance/4/return-risk/expected-return.aspx 
def generate_new_portfolio(prev_allocations, prev_returns, prev_sigmas):
    weights = np.random.random(len(ticker_symbols))
    weights /= np.sum(weights)
    allocations = prev_allocations + [weights]
    returns = prev_returns + [np.dot(weights, expected_return)]
    std_devs = prev_sigmas + [np.sqrt(np.dot(weights.T, np.dot(cov_return, weights)))] 
    return (allocations, returns, std_devs)

for i in range(1000):
    allocations, returns, std_devs = generate_new_portfolio(allocations, returns, std_devs)

In [13]:
p = figure(plot_width=900, plot_height=500)
p.scatter(x = std_devs, y = returns)
show(p)